In [ ]:
!pip3 install scikit-learn
!pip3 install imageio
!pip3 install IPython

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
import tensorflow.python.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from numpy import save, load

from sklearn.metrics import (
    average_precision_score,
    precision_recall_curve,
    roc_auc_score,
    roc_curve,
)

In [ ]:
import os

# tensorflow.keras.backend.set_floatx('float16')

PATCH_SIZE_X=32
PATCH_SIZE_Y=32
PATCH_SIZE_Z=32

SampleX = []
SampleY = []
LabelY = []

category_1_count = 0
category_2_count = 0
category_3_count = 0
category_4_count = 0
total_count = 0

def weighted_categorical_crossentropy(weights):
    weights = K.variable(weights)        
    def loss(y_true, y_pred):
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss    
    return loss

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (
                K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
                    
def soft_dice_loss(y_true, y_pred):
    print("y_true")
    print(y_true.shape)
    print("y_pred")
    print(y_pred.shape)
    # dice_loss = 1 - (1/4 * (dice_coef(y_true[0][0],y_pred[0][0])+dice_coef(y_true[0][1],y_pred[0][1])+dice_coef(y_true[0][2],y_pred[0][2])+dice_coef(y_true[0][3],y_pred[0][3])))
    dice_loss = 1 - (1/3 * (dice_coef(y_true[0][0],y_pred[0][0])+dice_coef(y_true[0][1],y_pred[0][1])+dice_coef(y_true[0][2],y_pred[0][2])))
    return dice_loss

# datasets=['model1batch1/']
# datasets=['model1batch2/'] - best 
# datasets=['model1batch3/']
# datasets=['model1batch4/'] 
# datasets=['modl1batch5/'] - worst
# datasets=['model1batch6/']

# datasets=['model1batch7/']
# datasets = ['model1batch8/']
# datasets = ['model1batch9/']
# datasets=['model1batch10/']
# datasets=['model1batch11/']

datasets = []
for dataset in datasets:
    for patch_file in os.listdir('../input/'+dataset):
        #filling in my input layer
        if "nifti" in patch_file:
            image = np.load('../input/'+dataset+patch_file)
            # train_x ~ empty 4 channel input tensor
            train_x = np.empty((4,PATCH_SIZE_X,PATCH_SIZE_Y,PATCH_SIZE_Z))
            # patient ID and patch number
            patient_id = patch_file.split('-')[1]
            patch_num = patch_file.split('-')[2].split(".")[0]
            ngtdm = 0
            
            # we've gone from patch creation --> numpy files --> train_x
            for x in range(0,PATCH_SIZE_X):
                for y in range(0,PATCH_SIZE_Y):
                    for z in range(0,PATCH_SIZE_Z):
                        train_x[0][x][y][z] = np.uint8(image[0][x][y][z])
                        train_x[1][x][y][z] = np.uint8(image[1][x][y][z])
                        train_x[2][x][y][z] = image[2][x][y][z]
                        train_x[3][x][y][z] = image[3][x][y][z]
                        total_count = total_count + 1
            SampleX.append(train_x)
            # print('image shape')
            # print(image.shape)
        
        if "label" in patch_file:
            label = np.load('../input/'+dataset+patch_file)
            
            # create label dataset ~ 1x32x32x32
            train_y = np.empty((4,PATCH_SIZE_X,PATCH_SIZE_Y,PATCH_SIZE_Z))
            patient_id = patch_file.split('-')[1]
            patch_num = patch_file.split('-')[2].split(".")[0]
            for x in range(0,PATCH_SIZE_X):
                for y in range(0,PATCH_SIZE_Y):
                    for z in range(0,PATCH_SIZE_Z):
                        if label[x][y][z] == 1:
                            train_y[0][x][y][z] = 1
                            train_y[1][x][y][z] = 0
                            train_y[2][x][y][z] = 0
                            train_y[3][x][y][z] = 0
                            category_1_count = category_1_count + 1
                        elif label[x][y][z] == 2:
                            # print("category (%d,%d,%d) 1" % (x,y,z))
                            train_y[0][x][y][z] = 0
                            train_y[1][x][y][z] = 1
                            train_y[2][x][y][z] = 0
                            train_y[3][x][y][z] = 0
                            category_2_count = category_2_count + 1
                        elif label[x][y][z] == 3:
                            # print("category (%d,%d,%d) 3" % (x,y,z))
                            train_y[0][x][y][z] = 0
                            train_y[1][x][y][z] = 0
                            train_y[2][x][y][z] = 1
                            train_y[3][x][y][z] = 0
                            category_3_count = category_3_count + 1
                        else:
                            # print("category (%d,%d,%d) 2" % (x,y,z))
                            train_y[0][x][y][z] = 0
                            train_y[1][x][y][z] = 0
                            train_y[2][x][y][z] = 0
                            train_y[3][x][y][z] = 1
                            category_4_count = category_4_count + 1
            # append all patches into a patch array (list of patches)
            # sampleY ~ for 5 fold cross validation
            LabelY.append(label)
            SampleY.append(train_y)


from keras.utils.np_utils import to_categorical
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

model=tensorflow.keras.models.load_model('../input/d/datasets/sokrishn/segmentationmodel/segmentation.model',custom_objects={'soft_dice_loss':soft_dice_loss})
model.save('./segmentation.model')

In [ ]:

import tensorflow
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Conv3DTranspose
from tensorflow.keras.optimizers import Adam
from keras.layers.merge import concatenate

keras.backend.set_image_data_format("channels_first")

In [ ]:
# first layer
input_layer = Input(shape=(4,32,32,32))
input_layer

# first down convolution 
down_depth_0_layer_0 = keras.layers.Conv3D(filters=32, 
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1),
                              data_format='channels_first',
                              input_shape=(4,32,32,32)
                              )(input_layer)


In [ ]:
# first down convolution - activation layer 1
down_depth_0_layer_0 = Activation('relu')(down_depth_0_layer_0)


In [ ]:
# first down convolution - activation layer 1
# Create a Conv3D layer with 64 filters and add relu activation
down_depth_0_layer_1 = Conv3D(filters=64, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_0_layer_0)
down_depth_0_layer_1 = Activation('relu')(down_depth_0_layer_1)

In [ ]:
# Define a max pooling layer
down_depth_0_layer_pool = MaxPooling3D(pool_size=(2,2,2),data_format='channels_first')(down_depth_0_layer_1)
down_depth_0_layer_pool

In [ ]:
# Add a Conv3D layer to your network with relu activation
down_depth_1_layer_0 = Conv3D(filters=64, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_0_layer_pool)
down_depth_1_layer_0 = Activation('relu')(down_depth_1_layer_0)

In [ ]:
# Add another Conv3D with 128 filters to your network.
down_depth_1_layer_1 = Conv3D(filters=128, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_1_layer_0)
down_depth_1_layer_1 = Activation('relu')(down_depth_1_layer_1)

In [ ]:
# Add an upsampling operation to your network
up_depth_0_layer_0 = UpSampling3D(size=(2,2,2))(down_depth_1_layer_1)
up_depth_0_layer_0

In [ ]:
# Print the shape of layers to concatenate
print(up_depth_0_layer_0)
print(down_depth_0_layer_1)

In [ ]:
# Add a concatenation along axis 1
up_depth_1_concat = concatenate([up_depth_0_layer_0,
                                 down_depth_0_layer_1],
                                axis=1)
up_depth_1_concat

In [ ]:
down_depth_0_layer_1
# Add a Conv3D up-convolution with 64 filters to your network
up_depth_1_layer_1 = keras.layers.Conv3D(filters=64, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_concat)
up_depth_1_layer_1 = keras.layers.Activation('relu')(up_depth_1_layer_1)

In [ ]:
# Add a Conv3D up-convolution with 64 filters to your network
up_depth_1_layer_2 = keras.layers.Conv3D(filters=64, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_layer_1)
up_depth_1_layer_2 = keras.layers.Activation('relu')(up_depth_1_layer_2)

In [ ]:
# Add a final Conv3D with 3 filters to your network.
final_conv = keras.layers.Conv3D(filters=4, #4 categories 
                    kernel_size=(1,1,1),
                    padding='valid',
                    strides=(1,1,1)
                    )(up_depth_1_layer_2)
final_conv

In [ ]:
# Add a sigmoid activation to your final convolution.
final_activation = keras.layers.Activation('sigmoid')(final_conv)
final_activation


In [ ]:
model = tensorflow.keras.models.Model(inputs=input_layer, outputs=final_activation)
            
model.compile(optimizer=tensorflow.keras.optimizers.Adam(lr=0.00001),
              loss=soft_dice_loss,
              metrics=['categorical_accuracy']
             )
model.summary()

num_patches=len(SampleX)
n1,n2,n3,n4 = SampleX[0].shape
InputX = np.array(SampleX)
InputX = InputX.reshape((num_patches,n1*n2*n3*n4))
n1,n2,n3 = LabelY[0].shape
InputY = np.array(LabelY)
InputY = InputY.reshape((num_patches,n1*n2*n3))

iteration = 0
kfold = KFold(n_splits=5, shuffle=True)
for train, test in kfold.split(InputX, InputY):
    trainX = np.array(SampleX)[train]
    trainY = np.array(SampleY)[train]
    validationX = np.array(SampleX)[test]
    validationY = np.array(SampleY)[test]
    model.fit(trainX,trainY,batch_size=1,epochs=100)
    iteration = iteration + 1

from keras.utils.np_utils import to_categorical

probY = model.predict(np.array(validationX))
probY = to_categorical(np.argmax(probY,axis=1), 4)
predY = probY.tolist()

from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

width=PATCH_SIZE_X
height=PATCH_SIZE_Y
depth=PATCH_SIZE_Z

Y_True = []
for v in validationY:
    for x in range(0,width):
        for y in range(0,height):
            for z in range(0,depth):
                for i in range(0,4):
                    if (v[i][x][y][z] == 1):
                        Y_True.append(i)
                        break

Y_Pred = []
for p in predY:
    for x in range(0,width):
        for y in range(0,height):
            for z in range(0,depth):
                for i in range(0,4):
                    if (p[x][y][z][i] == 1):
                        Y_Pred.append(i)
                        break
    
print("Confusion Matrix: ")
print(multilabel_confusion_matrix(np.array(Y_True), np.array(Y_Pred)))
print('Precision, Recall, F1 Score: ')
print(precision_recall_fscore_support(np.array(Y_True), np.array(Y_Pred), average='weighted', labels=np.unique(Y_Pred)))
print("model fit done")
print(model.metrics)

print("saving model")
model.save('./segmentation.model')
print("model save done")